In [15]:
import pandas as pd
import tkinter as tk
from tkinter import messagebox
from tkinter import ttk

# Load EV data
ev_data = pd.read_csv('C:\\Users\\Brylle\\Desktop\\Jupyter Projects\\pokemon-ev-tracker\\data\\pokemon_ev_yields.csv')
ev_data.columns = ev_data.columns.str.strip()

# Track EVs
ev_total = {
    "HP": 0,
    "Attack": 0,
    "Defense": 0,
    "Sp. Atk": 0,
    "Sp. Def": 0,
    "Speed": 0
}
MAX_TOTAL_EVS = 510
MAX_STAT_EVS = 255
BAR_LENGTH = 400  # Length of canvas bars

pokemon_list = sorted(ev_data['Pokemon'].tolist())

# Functions
def update_display():
    total = sum(ev_total.values())
    remaining_total = MAX_TOTAL_EVS - total
    ev_text = (
        f"HP: {ev_total['HP']} (Remaining: {MAX_STAT_EVS-ev_total['HP']})   "
        f"Attack: {ev_total['Attack']} (Remaining: {MAX_STAT_EVS-ev_total['Attack']})\n"
        f"Defense: {ev_total['Defense']} (Remaining: {MAX_STAT_EVS-ev_total['Defense']})   "
        f"Sp. Atk: {ev_total['Sp. Atk']} (Remaining: {MAX_STAT_EVS-ev_total['Sp. Atk']})\n"
        f"Sp. Def: {ev_total['Sp. Def']} (Remaining: {MAX_STAT_EVS-ev_total['Sp. Def']})   "
        f"Speed: {ev_total['Speed']} (Remaining: {MAX_STAT_EVS-ev_total['Speed']})\n"
        f"Total EVs: {total}/{MAX_TOTAL_EVS} (Remaining: {remaining_total})"
    )
    status_label.config(text=ev_text)
    
    # Update canvas bars
    for stat, canvas in canvas_bars.items():
        canvas.delete("bar")  # Clear previous bar
        percent = ev_total[stat] / MAX_STAT_EVS
        fill_length = percent * BAR_LENGTH
        # Determine color
        if percent < 0.4:
            color = "green"
        elif percent < 0.8:
            color = "yellow"
        else:
            color = "red"
        canvas.create_rectangle(0, 0, fill_length, 20, fill=color, tags="bar")

def add_single_pokemon_ev(entry):
    pokemon_name = entry.get().title()
    if pokemon_name == "" or pokemon_name == "Select Pokémon":
        return
    if pokemon_name not in ev_data['Pokemon'].values:
        messagebox.showerror("Error", f"Pokémon '{pokemon_name}' not found!")
        return
    
    row = ev_data[ev_data['Pokemon'] == pokemon_name].iloc[0]
    for stat in ev_total.keys():
        ev_total[stat] += row[stat]
        if ev_total[stat] > MAX_STAT_EVS:
            ev_total[stat] = MAX_STAT_EVS
    
    if sum(ev_total.values()) > MAX_TOTAL_EVS:
        messagebox.showwarning("Warning", "Total EVs exceeded 510!")
    
    update_display()

def reset_evs():
    for stat in ev_total.keys():
        ev_total[stat] = 0
    update_display()

# Autocomplete for Combobox
def autocomplete(event, combobox):
    typed = combobox.get()
    if typed == "":
        combobox['values'] = pokemon_list
    else:
        filtered = [p for p in pokemon_list if p.lower().startswith(typed.lower())]
        combobox['values'] = filtered

# GUI Setup
root = tk.Tk()
root.title("Pokémon EV Counter")
root.geometry("600x550")
root.resizable(False, False)

# Input Frame
input_frame = tk.Frame(root, pady=10)
input_frame.pack()

# Pokémon 1
tk.Label(input_frame, text="Pokémon 1:").grid(row=0, column=0, padx=5, sticky='w')
entry1 = ttk.Combobox(input_frame, values=pokemon_list)
entry1.grid(row=0, column=1, padx=5)
entry1.set("Select Pokémon")
entry1.bind("<KeyRelease>", lambda e: autocomplete(e, entry1))
tk.Button(input_frame, text="Add EVs 1", command=lambda: add_single_pokemon_ev(entry1)).grid(row=0, column=2, padx=5)

# Pokémon 2
tk.Label(input_frame, text="Pokémon 2:").grid(row=1, column=0, padx=5, sticky='w')
entry2 = ttk.Combobox(input_frame, values=pokemon_list)
entry2.grid(row=1, column=1, padx=5)
entry2.set("Select Pokémon")
entry2.bind("<KeyRelease>", lambda e: autocomplete(e, entry2))
tk.Button(input_frame, text="Add EVs 2", command=lambda: add_single_pokemon_ev(entry2)).grid(row=1, column=2, padx=5)

# Pokémon 3
tk.Label(input_frame, text="Pokémon 3:").grid(row=2, column=0, padx=5, sticky='w')
entry3 = ttk.Combobox(input_frame, values=pokemon_list)
entry3.grid(row=2, column=1, padx=5)
entry3.set("Select Pokémon")
entry3.bind("<KeyRelease>", lambda e: autocomplete(e, entry3))
tk.Button(input_frame, text="Add EVs 3", command=lambda: add_single_pokemon_ev(entry3)).grid(row=2, column=2, padx=5)

# EV Display
status_label = tk.Label(root, text="", justify="left", font=("Helvetica", 12))
status_label.pack(pady=10)

# Canvas bars
canvas_frame = tk.Frame(root)
canvas_frame.pack(pady=10)
canvas_bars = {}
for idx, stat in enumerate(ev_total.keys()):
    tk.Label(canvas_frame, text=stat).grid(row=idx, column=0, sticky='w')
    canvas = tk.Canvas(canvas_frame, width=BAR_LENGTH, height=20, bg="lightgray")
    canvas.grid(row=idx, column=1, padx=5, pady=2)
    canvas_bars[stat] = canvas

# Reset Button
tk.Button(root, text="Reset EVs", command=reset_evs).pack(pady=10)

# Initialize display
update_display()

root.mainloop()
